# Qubit Tapering 
### in the Stabilizer Subspace Projection formalism
Here, we take a look at the qubit reduction technique of [tapering](https://arxiv.org/abs/1701.08213) and an implementation based on the core `S3_projection` class. Unlike [Contextual-Subspace VQE](https://doi.org/10.22331/q-2021-05-14-456), this technique is *exact*, in the sense that it perfectly preserves the energy spectrum of the input operator.

At the core of qubit tapering is a symmetry of the Hamiltonian, which in this case means a set of universally commuting operators. The idea is that these operators must be simultaneously measureable and so can be treated independently of the remaining Hamiltonian terms. The method works by finding an independent generating set for the symmetry and seeks to find the 'correct' assignment of eigenvalues (called a *sector*), which completely determines the measurement outcome of the symmetry operators. Once this is obtained, the theory of stabilizers allows us to rotate the symmetry generators onto single Pauli $X$ operators, and since they must commute universally every operator of the rotated Hamiltonian will consist of an identity or Pauli $X$ in the corresponding qubit position. This means we can drop the qubit from the Hamiltonian, leaving in its place the eigenvalue determined by the chosen sector.

In [23]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import openfermion as of
import os
import json
from symred.symplectic_form import PauliwordOp, QubitHamiltonian
from symred.S3_projection import QubitTapering

def get_ground_state(sparse_operator, initial_guess=None):
    """Compute lowest eigenvalue and eigenstate.
    Args:
        sparse_operator (LinearOperator): Operator to find the ground state of.
        initial_guess (ndarray): Initial guess for ground state.  A good
            guess dramatically reduces the cost required to converge.
    Returns
    -------
        eigenvalue:
            The lowest eigenvalue, a float.
        eigenstate:
            The lowest eigenstate in scipy.sparse csc format.
    """
    values, vectors = sp.sparse.linalg.eigsh(sparse_operator,
                                            k=1,
                                            v0=initial_guess,
                                            which='SA',
                                            maxiter=1e7)

    order = np.argsort(values)
    values = values[order]
    vectors = vectors[:, order]
    eigenvalue = values[0]
    eigenstate = vectors[:, 0]
    return eigenvalue, eigenstate.T


def plot_ground_state_amplitudes(eigenvalue,eigenvector, 
                                num_qubits: int, 
                                reverse_bitstrings: bool=False,
                                return_amps: bool = False,
                                matrix_type='sparse'
                                )-> None:
    """ Prints a barplot of the probability amplitudes for each 
    basis state in the ground eigenstate of the input operator
    """
    bitstrings = [format(index, f'0{num_qubits}b') for index in range(2**(num_qubits))]
    if reverse_bitstrings:
        bitstrings.reverse()
    amps = [(b_str, amp) for b_str,amp 
            in zip(bitstrings, np.square(abs(eigenvector))) if amp>1e-5]
    amps = sorted(amps, key=lambda x:-x[1])
    X, Y = zip(*amps)
    
    # plot and show the amplitudes
    plt.bar(X, Y)
    plt.xlabel('Basis state')
    plt.ylabel('Amplitude in ground state')
    plt.title(f'Energy = {eigenvalue: .10f}')
    plt.xticks(rotation=90)
    plt.show()

    if return_amps:
        return amps
    

First, we shall construct a molecule using OpenFermion with PySCF the underlying quantum chemistry package. The resulting fermionic operator will be mapped onto qubits via the Jordan-Wigner transformation.

In [2]:
working_dir = os.getcwd()
parent_dir = os.path.dirname(working_dir)
data_dir = os.path.join(parent_dir,'data')

In [ ]:
# os.listdir(data_dir)

In [3]:
file = 'O1_STO-3G_triplet_OO.json'
# file = 'H2-Be1_STO-3G_singlet_BeH2BeH2.json'

file_path = os.path.join(data_dir, file)
with open(file_path, 'r') as input_file:
    ham_dict = json.load(input_file)
    
ham = QubitHamiltonian(ham_dict)
ham

We are now in a position to initialize our `QubitTapering` class, which will identify a set of independent operators that generate the Hamiltonian symmetry.

In [4]:
taper_hamiltonian = QubitTapering(ham)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand are rotated onto the single-qubit Pauli operators\n')
print(taper_hamiltonian.rotated_stabilizers)
print('\nvia a sequence of Clifford pi/2 rotations\n')
print(*taper_hamiltonian.rotations)

We are able to taper 5 qubits from the Hamiltonian.

The symmetry generators are

(1+0j) ZIIZIZIZIZ +
(1+0j) IZIZIZIZIZ +
(1+0j) IIZZIIIIII +
(1+0j) IIIIZZIIII +
(1+0j) IIIIIIZZZZ

and are rotated onto the single-qubit Pauli operators

(-1+0j) IIIIIIXIII +
(-1+0j) IIIIXIIIII +
(-1+0j) IIXIIIIIII +
(-1+0j) IXIIIIIIII +
(-1+0j) XIIIIIIIII

via a sequence of Clifford pi/2 rotations

YIIZIZIZIZ IYIZIZIZIZ IIYZIIIIII IIIIYZIIII IIIIIIYZZZ


In order to perform the stabilizer subspace projection, we must also supply a symmetry sector or reference state. Under the Jordan-Wigner transformation, the Hartree-Fock state for our $M$-electron, $N$-orbital molecular system with charge=0 and multiplicity=1 will be 

$$|\mathrm{HF}\rangle = |\underbrace{1 \dots 1}_{M \,\text{times}}\; \underbrace{0 \dots 0}_{N-M \,\text{times}} \rangle.$$

Note that OpenFermion fills orbital occupations from the left... this will not always be the case! For example, if using Qiskit or some other quantum computing package the Hartree-Fock state will not look the same.

In [5]:
n_electrons = 6
n_qubits = len(list(ham_dict.keys())[0])

In [18]:
ham.Get_ground_state(n_eig_vals=1)

true_gs_energy, true_gs_vec = ham.eig_vals[0], ham.eig_vecs[0,:]
guess_state = np.binary_repr(np.argmax(true_gs_vec), width=n_qubits)

In [19]:
# hf_state = np.concatenate([np.ones(n_electrons, dtype=int),np.zeros(n_qubits-n_electrons, dtype=int)])
# hf_string = ''.join([str(i) for i in hf_state])

hf_string = guess_state
print(f'The Hartree-Fock state is |{hf_string}>')

hf_state = np.array(list(map(int, hf_string)))

The Hartree-Fock state is |0101111111>


The corresponding sector is obtained by measuring each symmetry generator with respect to the reference state, yielding a $\pm1$ eigenvalue assignment.

In [20]:
print(f'The symmetry sector corresponding with the reference state is {taper_hamiltonian.identify_symmetry_sector(hf_state)}')

The symmetry sector corresponding with the reference state is [ 1 -1 -1  1  1]


This is everything we need to go ahead and perform the tapering process, which is effected by the `taper_it()` method that calls on the parent `S3_projection` class.

In [21]:
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_state)
print('Tapered Hamiltonian:\n\n', ham_tap)

Tapered Hamiltonian:

 (-47.24319175857829+0j) IIIII +
(12.420195218611381+0j) IIIIZ +
(12.420195218611381+0j) IIIZI +
(1.386087206475665+0j) IIIZZ +
(1.6560274461609374+0j) IIZII +
(0.520181883024243+0j) IIZIZ +
(0.5570302427543667+0j) IIZZI +
(1.6560274461609377-0j) IIZZZ +
(2.6678191376918603+0j) IZIII +
(0.5512397458758697+0j) IZIIZ +
(0.5512397458758697+0j) IZIZI +
(0.3586246955560466+0j) IZZII +
(0.023722222559192097+0j) IZZIZ +
(0.3586246955560466+0j) IZZZZ +
(2.220446049250313e-16+0j) ZIIII +
(-0.00645688621584839+0j) ZIIIZ +
(0.00645688621584839+0j) ZIIZI +
(-0.043093593380162915+0j) ZIZII +
0j ZIZIZ +
(0.043093593380162915+0j) ZIZZZ +
0j ZZIII +
(0.043093593380162915+0j) ZZIIZ +
(-0.043093593380162915+0j) ZZIZI +
(0.00645688621584839+0j) ZZZII +
(-4.440892098500626e-16+0j) ZZZIZ +
(-0.00645688621584839+0j) ZZZZZ +
(-0.00896331655087819+0j) IIIIX +
(0.00896331655087819-0j) IIZZX +
(0.00896331655087819-0j) ZZIZX +
(-0.00896331655087819+0j) ZZZIX +
(-0.11259375353764486+0j) IIIX

We should also check that the ground state energy of the tapered Hamiltonian mathces that of the full system.

In [25]:
# true_gs_energy, true_gs_vec = get_ground_state(ham.to_sparse_matrix)
tap_gs_energy, tap_gs = get_ground_state(ham_tap.to_sparse_matrix)

print(f'The ground state energy of the full system is {true_gs_energy},')
print(f'whereas for the tapered system we find the energy is {tap_gs_energy}.')
print(f'The absolute error is {tap_gs_energy-true_gs_energy}.')

The ground state energy of the full system is -73.80415023325718,
whereas for the tapered system we find the energy is -73.8041502332558.
The absolute error is 1.3784529073745944e-12.


Do they match? Depending on the molecule chosen, they might not! One can sometimes find that the Hartree-Fock state does not yield the correct symmetry sector, particularly in the strongly correlated regime.

In [26]:
hf_vec = np.eye(1,len(true_gs_vec),int(hf_string,2))
hf_overlap = np.square(np.abs(hf_vec.dot(true_gs_vec)))[0]

if hf_overlap < 1e-18:
    print('The Hartree-Fock state has no overlap with the true ground state!')
else:
    print('The Hartree-Fock state exhibits non-zero overlap with the true ground state.')
print(f'Overlap of the Hartree-Fock state with the true ground state: <HF|True GS> = {hf_overlap:.10f}')

The Hartree-Fock state exhibits non-zero overlap with the true ground state.
Overlap of the Hartree-Fock state with the true ground state: <HF|True GS> = 0.1119320479


If we instead take the dominant basis state (plotted below in a histogram), we should see that the energies do match in the resulting sector...

In [ ]:
amps = plot_ground_state_amplitudes(true_gs_energy,true_gs_vec, n_qubits, return_amps=True)
new_ref_state = [int(i) for i in amps[0][0]]
ham_tap_2 = taper_hamiltonian.taper_it(ref_state=new_ref_state)
tap_gs_energy_2, tap_gs_2 = get_ground_state(ham_tap_2.to_sparse_matrix)

print(f'In the sector {taper_hamiltonian.identify_symmetry_sector(new_ref_state)}, we find the ground state energy to be {tap_gs_energy_2}.')
print(f'The absolute error is {tap_gs_energy_2-true_gs_energy}.')

*The problem is...* 

we will not in general know how the basis states are distributed in the ground state!

The scalability of tapering is highly predicated on finding new approaches to identifying the correct symmetry sector.

In [ ]:
from openfermion.ops import MajoranaOperator

In [ ]:
y1 = MajoranaOperator(term=(1,), coefficient=1.0)
y2 = MajoranaOperator(term=(2,), coefficient=1.0)
y1*y2 == -1*y2*y1

In [ ]:
y1 = MajoranaOperator(term=(1,2,3,4), coefficient=1.0)
y2 = MajoranaOperator(term=(1,2,3), coefficient=1.0)

y1*y2 == - y2*y1

In [ ]:
class MajoranaOp:
    """ 
    A class thats represents an operator defined over the Pauli group in the symplectic representation.
    """
    def __init__(self, 
            n_qubits, list_lists_OR_sympletic_form
        ) -> None:
        """ 
        TODO
        """
        
        self.n_qubits = n_qubits
        self.initalize_op(list_lists_OR_sympletic_form)
        
    def initalize_op(self, input_term):
        
        if isinstance(input_term, np.ndarray):
            assert(input_term.shape[1] == 2*self.n_qubits), 'incorrect qubit no.'
            self.m_op = input_term
        else:
            n_terms = len(input_term)
            self.m_op = np.zeros((n_terms, 2*self.n_qubits), dtype=int)
            for ind, term in enumerate(input_term):
                self.m_op[ind,term]=1
                
        
    def commutes(self, M_OP):
#         sum(np.logical_and(self.m_op, M_OP.m_op))
        return not bool(np.dot(self.m_op, M_OP.m_op)%2)
        
    def commutes_termwise(self, M_OP):
        # 1 means terms anticommute!
        # 0 means terms commute!
        comm_flag = np.dot(self.m_op, M_OP.m_op.T)%2
        return comm_flag
    
    def adjacency_matrix(self):
        """ Checks which terms of self commute within itself
        """
        adj = self.commutes_termwise(self)
        np.fill_diagonal(adj, 0) # zero means commutes (need to set diagonal!)
        return adj

In [ ]:
M = MajoranaOp(3, [[0,1],[1],[0,1,2],[2], [3]])
M.adjacency_matrix()


In [ ]:
from symred.S3_projection import gf2_basis_for_gf2_rref, gf2_gaus_elim
ZX_symp = M.m_op
reduced = gf2_gaus_elim(ZX_symp)
kernel  =  gf2_basis_for_gf2_rref(reduced)
kernel = np.logical_not(kernel).astype(int) # uses not as we get the anticommutator 
kernel

In [ ]:
from openfermion.linalg import get_sparse_operator
get_sparse_operator(y6, n_qubits=10)

In [ ]:
y6 = MajoranaOperator(term=(0,1,2,3,5), coefficient=1.0)
y7 = MajoranaOperator(term=(0,1,2,3,4), coefficient=1.0)

y_all = (MajoranaOperator(term=(0,1), coefficient=1.0) + 
        MajoranaOperator(term=(1,), coefficient=1.0) + 
        MajoranaOperator(term=(0,1,2), coefficient=1.0) + 
        MajoranaOperator(term=(2,), coefficient=1.0) )

print(y6 * y_all == y_all * y6)
print(y7 * y_all == y_all * y7)


In [ ]:


# def symmetry_generators(self) -> StabilizerOp:
#     """ Find an independent basis for the input operator symmetry
#     """
#     # swap order of XZ blocks in symplectic matrix to ZX
#     ZX_symp = np.hstack([self.operator.Z_block, self.operator.X_block])
#     reduced = gf2_gaus_elim(ZX_symp)
#     kernel  = gf2_basis_for_gf2_rref(reduced)

In [ ]:
M1 = MajoranaOp(12, [[1,2,3,4], [1,2]])
M2 = MajoranaOp(12, [[1,2,3],[1],[10],[2]])

M1.commutes_termwise(M2)
# first term in M1 anticomm, anticomm, commutes, anticomm with M2 terms
# second ....   M1 commutes, anticomm, commutes, anticomm with M2 terms

In [ ]:
y1 = MajoranaOperator(term=(1,2,3), coefficient=1.0)
y2 = MajoranaOperator(term=(1,2,3), coefficient=1.0)

y1*y2 == y2*y1

In [ ]:
#     @cached_property
#     def symmetry_generators(self) -> StabilizerOp:
#         """ Find an independent basis for the input operator symmetry
#         """
#         # swap order of XZ blocks in symplectic matrix to ZX
#         ZX_symp = np.hstack([self.operator.Z_block, self.operator.X_block])
#         reduced = gf2_gaus_elim(ZX_symp)
#         kernel  = gf2_basis_for_gf2_rref(reduced)

In [ ]:
M1.shape

In [ ]:
M1 = MajoranaOp(6, [[1,2,3,4]])
M2 = MajoranaOp(6, [[1,2]])

# M1.commutes(M2)


In [ ]:
y1 = MajoranaOperator(term=(1,2,3,4), coefficient=1.0)
y2 = MajoranaOperator(term=(1,2), coefficient=1.0)

y1*y2 == -y2*y1